In [ ]:
"""
Title: Pittingfactor Calculation

Usage: Used to calculate the Pitting factor of degraded wires for Micro-CT Image data
Input: 2D-Tif Image data 
Output: 2D/3D Pittingfactor, Mean Degradation Depth and Deepest Pit per slice

Written by: Sven Schimek, André Lopes Marinho
Contact. sven.schimek@hereon.de
Update: 08.07.24
Copyright: Helmholtz-Zentrum Hereon

"""

In [ ]:
#Definition of used functions
import numpy as np
import pyvista as pv
import os
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from scipy.ndimage import distance_transform_edt
import scipy as sp
import skimage.io
import skimage.viewer
from sklearn.metrics import jaccard_score
import ipympl


def load_image(path, smooth=False, labels_to_smooth=None):
    """Loads an image (or image stack) given a defined path.
    
    Args:
        path(string): String containing path for images with proper file extension. 
            Example: "images/*.tif"
    
    Returns: 
        numpy.ndarray: Array with images defined in path.
    """

    seq_img = skimage.io.imread_collection(load_pattern=path, conserve_memory=True)
    img_list = []
    num_img = 0 
    for image in seq_img:
        if smooth: # Todo: Verify error when labels_to_smooth=None
            smooth_img = smooth_labels(image=seq_img[num_img], labels=labels_to_smooth) 
            img_list.append(smooth_img)
        else:
            img_list.append(seq_img[num_img])

        num_img = num_img + 1 # After for loop, num_img will represent the stack size
    loaded_img = np.array(img_list)
    
    return loaded_img

def stack_2d_images(image_list, label1, label2):
          
    """Loads an image (or image stack) given a defined path.
    
    Args:
        image_list: numpy.ndarray with the loaded images
        label 1: Label of the Degradation Layer
        label 2: Label of the Bulk Material
    
    Returns: 
        numpy.ndarray: Array with stacked images.
    """
 
    if len(image_list) == 0:
        raise ValueError("The input image list is empty.")

    # Get the dimensions of the first image to determine the shape of the 3D array
    shape_2d = image_list[0].shape

    # Initialize a 3D numpy array filled with zeros
    stacked_image = np.zeros((len(image_list), shape_2d[0], shape_2d[1]), dtype=image_list[0].dtype)

    # Stack the 2D images along the first dimension to create a 3D array
    for i in range(len(image_list)):
        # Filter pixels based on labels and store in the stacked_image
        mask = (image_list[i] == label2)
        stacked_image[i, :, :] = np.where(mask, 2, 0)
        
    return stacked_image

    
   
    
def contact_area(image, label1, label2):
    """Counts the number of surfaces of label1 in contact with label2.
    
    Args:
        image(numpy.ndarray): Array representing image (or image stack). 
        label1(int): Value of label 1
        label1(int): Value of label 2
  
    Returns: 
        int: Number of voxels of label1 in contact with label2
    """
    
    histogram = skimage.exposure.histogram(image)
    
    if label1 not in histogram[1] or label2 not in histogram[1]:
        raise ValueError('One or more labels do not exist. Please input valid labels.') 
        
    x_contact_1 = np.logical_and(image[:, :, :-1] == label1, image[:, :, 1:] == label2)
    y_contact_1 = np.logical_and(image[:, :-1, :] == label1, image[:, 1:, :] == label2)
    z_contact_1 = np.logical_and(image[:-1, :, :] == label1, image[1:, :, :] == label2)

    x_contact_2 = np.logical_and(image[:, :, :-1] == label2, image[:, :, 1:] == label1)
    y_contact_2 = np.logical_and(image[:, :-1, :] == label2, image[:, 1:, :] == label1)
    z_contact_2 = np.logical_and(image[:-1, :, :] == label2, image[1:, :, :] == label1)

    x_surface = np.logical_or(np.logical_and(x_contact_1, np.logical_not(x_contact_2)),
                              np.logical_and(x_contact_2, np.logical_not(x_contact_1)))
    y_surface = np.logical_or(np.logical_and(y_contact_1, np.logical_not(y_contact_2)),
                              np.logical_and(y_contact_2, np.logical_not(y_contact_1)))
    z_surface = np.logical_or(np.logical_and(z_contact_1, np.logical_not(z_contact_2)),
                              np.logical_and(z_contact_2, np.logical_not(z_contact_1)))
    x_coordinates = np.argwhere(x_surface)
    y_coordinates = np.argwhere(y_surface)
    z_coordinates = np.argwhere(z_surface)
    coordinates=np.vstack((x_coordinates,y_coordinates,z_coordinates))
    coordinates_unique=np.unique(coordinates,axis=0)
   
    return coordinates_unique

In [ ]:
#Setting the Path's for the ref (nondegraded wire) and target path (degraded wire)
ref_path = '/asap3/petra3/gpfs/p05/2023/data/11018037/processed/0125_ae26/reco/Andre_DR/Pre/*.tif'
target_path = '/asap3/petra3/gpfs/p05/2023/data/11018037/processed/0125_ae26/reco/Andre_DR/Post/*.tif'

In [ ]:
#Extracting the surfaces of the sample for the Pitting-factor analysis

label1=3 #Degra_Layer
label2=2 #Bulk

#Loading the Image Data
image_list = load_image(ref_path)
stacked_3d_image = stack_2d_images(image_list,label1,label2)


# Calculating the midpoint of each slice based on the label coordinates
array_mittelpunkt = np.ones_like(stacked_3d_image)

for i, plane in enumerate(stacked_3d_image):
    indices_x, indices_y = np.where(plane == 2)
    if len(indices_x) > 0:
        midpoint_x = int(np.mean(indices_x))
        midpoint_y = int(np.mean(indices_y))
        array_mittelpunkt[i, midpoint_x, midpoint_y] = 0
        


# Generate a distance map based on the mid point for each slice
distance_map=np.zeros_like(stacked_3d_image,dtype=float)
for i in range(stacked_3d_image.shape[0]):
    distance_map[i,:,:] = distance_transform_edt(array_mittelpunkt[i,:,:])



# Extracting the surface based on the samlpe labels which are in contact with the background label
coordinates = []

coordinates=contact_area(image=stacked_3d_image,label1=0,label2=2)
              

# Calculate the distance between the surface pixels and the midpoint
surface_distance = np.zeros_like(stacked_3d_image,dtype=np.uint32)

for coord in coordinates:
    i, x, y = coord
    surface_distance[i, x, y] = distance_map[i, x, y]



#Repeat the procedure for the Target-path images

#Loading the Image Data
image_list_target = load_image(target_path)
stacked_3d_image_target = stack_2d_images(image_list_target,label1,label2)

# Extracting the surface based on the sample labels which are in contact with the background label
coordinates_target = []


coordinates_target=contact_area(image=stacked_3d_image_target,label1=0,label2=2)
            
#Calculate the distance between the surface pixels and the midpoint
surface_distance_target = np.zeros_like(stacked_3d_image_target,dtype=np.uint32)

for coord in coordinates_target:
    i, x, y = coord
    surface_distance_target[i, x, y] = distance_map[i, x, y]



In [ ]:
#Calculating the Pitting factor based on the surface_distance arrays
import numpy as np
from scipy.spatial.distance import cdist

#Set the pixelsize
pixel_size=0.291

#Setup parameters
points_pre = np.argwhere(surface_distance > 0)
points_post = np.argwhere(surface_distance_target > 0)
num_layers = surface_distance.shape[0]
mean_diff_total=[]
mean_diff_total_std=[]
max_depth_total=[]
pittingfactor_per_level=[]
pittingfactor_mean=0
pittingfactor_std=0
delta_difference_target = np.zeros_like(surface_distance_target)
pittingfactors=[]
differences_per_layer=[]
x_layer_total=[]



#Calculation of the Pittingfactor: Loop for all layers
for x_layer in range(num_layers):
    points_pre_layer=points_pre[points_pre[:,0]==x_layer]
    points_post_layer=points_post[points_post[:,0]==x_layer]
    # Use the cdist function to find for the ref_path points the closest points in the target_path image
    distances = cdist(points_post_layer, points_pre_layer)
    closest_point_indices = np.argmin(distances, axis=1)
    closest_points = points_pre_layer[closest_point_indices]

    # Calculate the difference between the points
    differences=[]
    

    for i in range(points_post_layer.shape[0]):
        y, z = points_post_layer[i,1:]
        y_target, z_target = closest_points[i,1:]
        diff = surface_distance[x_layer, y_target, z_target] - surface_distance_target[x_layer, y, z]
        diff_mm=diff*pixel_size
        differences.append(diff_mm)
        delta_difference_target[x_layer,y,z] = diff_mm
        differences_per_layer.append(differences)
        mean_diff=np.mean(differences)
        mean_diff_std=np.std(differences)
    if mean_diff==0:
        pittingfactor_per_level=1
    else:
        max_depth=np.max(differences)
        pittingfactor_per_level=(max_depth/mean_diff)
    pittingfactors.append(pittingfactor_per_level)
    x_layer_total.append(x_layer)
    mean_diff_total_std.append(mean_diff_std)
    mean_diff_total.append(mean_diff)
    max_depth_total.append(max_depth)
    pittingfactors.append(pittingfactor_per_level)
    del distances, closest_point_indices, closest_points,differences

pittingfactor_mean=np.mean(pittingfactors)
pittingfactor_std=np.std(pittingfactors)



In [ ]:
#Calculation of the parameters
MaxDepth=np.max(max_depth_total)
MeanDiff=np.mean(mean_diff_total)
MeanDiff_std=np.std(mean_diff_total)
print(f"pittingfaktor_2d {pittingfactor_mean}")
print(f"pittingfaktor_2d_std {pittingfactor_std}")
print(f"max_depth:{MaxDepth}")
print(f"mean diff:{MeanDiff}")
print(f"STD mean diff:{MeanDiff_std}")
print(f"3D Pittingfaktor: {MaxDepth/MeanDiff}")

In [ ]:
#Saving of the values

from openpyxl import Workbook



# Create a new workbook
wb = Workbook()

# Select the active worksheet
ws = wb.active


ws.cell(row=1, column=1, value="Mean Diff")

# Write the list data to the worksheet
for index, value in enumerate(mean_diff_total, start=2):
    ws.cell(row=index, column=1, value=value)
    
ws.cell(row=1, column=2, value="Mean Diff STD")
for index, value in enumerate(mean_diff_total_std, start=2):
    ws.cell(row=index, column=2, value=value)

ws.cell(row=1, column=3, value="Max Depth")

for index, value in enumerate(max_depth_total, start=2):
    ws.cell(row=index, column=3, value=value)

ws.cell(row=1, column=4, value="PF-perLayer")

for index, value in enumerate(pittingfactors, start=2):
    ws.cell(row=index, column=4, value=value)

ws.cell(row=1, column=5, value="Layer")

for index, value in enumerate(x_layer_total, start=2):
    ws.cell(row=index, column=5, value=value)    
    
# Specify the directory path where you want to save the file
directory_path = ""

# Save the workbook to a file in the specified directory
wb.save(directory_path + "RawData_PfScript.xlsx")
